# Financial Analyst Agent Using CrewAI and LlamaIndex

### Install required dependencies

In [1]:
!pip install llama-index --quiet
!pip install llama-index-llms-groq --quiet
!pip install llama-index-core --quiet
!pip install llama-index-readers-file --quiet
!pip install llama-index-tools-wolfram-alpha --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install 'crewai[tools]' --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/2

###  Setup the LLM with Groq.

In [2]:
from llama_index.llms.groq import Groq
llm = Groq(model="llama3-70b-8192", api_key="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")

In [3]:
response = llm.complete("Explain the importance of LLMs")


### chat based model for binding

In [4]:
from langchain_openai import ChatOpenAI

# Define your API key here
groq_api_key = "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"

chat_llm = ChatOpenAI(

    openai_api_key=groq_api_key,
    model="mixtral-8x7b-32768",
    openai_api_base="https://api.groq.com/openai/v1",
    temperature=0,
    max_tokens=1000,
)

In [5]:
response = chat_llm("what is ai agent  ")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [6]:
response

AIMessage(content="An AI agent, or artificial intelligence agent, is a software program that is designed to perform tasks that would normally require human intelligence. These tasks can include things like learning, problem-solving, decision-making, and perception.\n\nAI agents can be categorized based on their level of autonomy and the type of environment in which they operate. For example, a simple AI agent might be a rule-based system that follows a predetermined set of instructions to complete a specific task. A more complex AI agent might be a machine learning model that is trained on data and can make decisions based on patterns it has learned.\n\nAI agents can be used in a variety of applications, such as customer service chatbots, autonomous vehicles, and personal assistants. They can also be used in research and development to simulate and study complex systems.\n\nIt's important to note that AI agents are not sentient or conscious in the way that humans are. They do not have 

### Download data

In [7]:
!wget "https://www.infosys.com/investors/reports-filings/annual-report/annual/documents/infosys-ar-23.pdf" -O infy_nav.pdf

--2024-08-01 09:40:15--  https://www.infosys.com/investors/reports-filings/annual-report/annual/documents/infosys-ar-23.pdf
Resolving www.infosys.com (www.infosys.com)... 184.28.52.227
Connecting to www.infosys.com (www.infosys.com)|184.28.52.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘infy_nav.pdf’

infy_nav.pdf            [ <=>                ]  12.52M  70.0MB/s    in 0.2s    

2024-08-01 09:40:15 (70.0 MB/s) - ‘infy_nav.pdf’ saved [13133646]



### Parse the content of the data

In [8]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
import os
from langchain_openai import ChatOpenAI

In [9]:
reader = SimpleDirectoryReader(input_files=["infy_nav.pdf"])
docs = reader.load_data()
docs[1]

Document(id_='35fc9aee-f286-4e00-bd9d-6b2129121008', embedding=None, metadata={'page_label': '2', 'file_name': 'infy_nav.pdf', 'file_path': 'infy_nav.pdf', 'file_type': 'application/pdf', 'file_size': 13133646, 'creation_date': '2024-08-01', 'last_modified_date': '2023-06-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The context surrounding \nan enterprise, created, \nand influenced by multiple \ninherently uncertain forces, can \nsignificantly impact the fortunes \nof a business. While this volatility \nhas come to be expected as normal, \nnot every business emerges from it having \ntackled the situation with the same resilience. \nThose that get a head start in preparing and taking \non the challenges are also the ones that come out of the \nun

###  embedding model

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Build Index

In [11]:
index = VectorStoreIndex.from_documents(docs,
                                        embed_model=embed_model,
                                        )

### Build the Query Engine

In [12]:
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)


### Instantiate the query engine as a Tool

In [13]:
from crewai_tools import LlamaIndexTool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="infy Query Tool",
    description="Use this tool to lookup the 2022-23 Infosys Annual Report",
)

In [14]:
query_tool.args_schema.schema()


{'title': 'QueryToolSchema',
 'description': 'Schema for query tool.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'Search query for the query tool.',
   'type': 'string'}},
 'required': ['query']}

### Instantiate Researcher and Writer Agents

In [15]:
import os
from crewai import Agent, Task, Crew, Process

In [16]:
researcher = Agent(
    role="Lead Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stocks like Infosys.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
    llm=chat_llm,
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    llm=chat_llm,
    verbose=True,
    allow_delegation=False,
)

### Define respective tasks

In [17]:
task1 = Task(
    description="""Conduct a comprehensive analysis of Infosys's use of artificial intelligence in their integrated annual report 2022-23.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the importance of artificial intelligence at Infosys.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

### Instantiate the Crew with a sequential process

In [18]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2,  # You can set it to 1 or 2 to different logging levels
)

### Initiate the agent workflow: Let the magic begin!

In [19]:
result = crew.kickoff()

print("######################")
print(result)

 [2024-08-01 09:44:38][DEBUG]: == Working Agent: Lead Financial Analyst
 [2024-08-01 09:44:38][INFO]: == Starting Task: Conduct a comprehensive analysis of Infosys's use of artificial intelligence in their integrated annual report 2022-23.


> Entering new CrewAgentExecutor chain...
To conduct a comprehensive analysis of Infosys's use of artificial intelligence (AI) in their integrated annual report 2022-23, I need to gather all the relevant information about their AI initiatives, investments, partnerships, and applications. I will use the infy Query Tool to search for the necessary information in the annual report.

Action: infy Query Tool

Action Input: {'query': 'Artificial Intelligence'}
 

In today's uncertain landscape, companies like Infosys are harnessing the power of Artificial Intelligence (AI) to drive flexibility and optionality in their operations. By adopting an AI-first approach, Infosys is not only transforming its own enterprise but also empowering its clients to navig